In [1]:
!pip install pandas scikit-learn optuna seaborn

In [2]:
import pandas as pd
import seaborn as sns
from include.utils import incrementar_dados_aleatorios_csv, categorizar_imc
import optuna
import os

In [3]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

**1. Engenharia de Recursos**

Novos recursos derivados dos dados existentes

In [4]:
# Obtém a largura do terminal - uso mais embaixo nos prints
terminal_width = os.get_terminal_size().columns

# leia os dados CSV indicados no site da FIAP
dados = pd.read_csv('../base_dados.csv', encoding='latin-1')

# aqui chamo a função incrementar_dados_aleatorios, para aumentar o volume da variável dados
dados = incrementar_dados_aleatorios_csv(dados)

# Aplicar a função para categorizar IMC e criar uma nova coluna 'Categoria IMC'
dados['Categoria IMC'] = dados['IMC'].apply(categorizar_imc)

# Mapeamento das regiões para valores numéricos
regiao_numerica = {'sudoeste': 1, 'sudeste': 2, 'nordeste': 3, 'noroeste': 4}

# Convertendo os valores da coluna 'Região' para valores numéricos usando o mapeamento
dados['Região_numerica'] = dados['Região'].map(regiao_numerica)

**2. Limpeza de Dados**

In [5]:
# Remover linhas com valores NaN
dados = dados.dropna()

# Substituir valores nulos por zero
dados['Filhos'] = dados['Filhos'].fillna(0)
dados['Filhos'] = dados['Filhos'].astype(int)
dados['Idade'] = dados['Idade'].fillna(0)
dados['Idade'] = dados['Idade'].astype(int)
dados['Encargos'] = dados['Encargos'].fillna(0)

dados['Gênero'] = dados['Gênero'].str.replace('nan', 'Não informado')

**3. Pré-processamento**

In [6]:
# Dividir os dados em features (X) e target (y)
X = dados.drop('Encargos', axis=1)  # Coluna alvo
y = dados['Encargos']

# Codificar variáveis categóricas usando one-hot encoding
encoder = OneHotEncoder(drop='first')
X_encoded = encoder.fit_transform(X[['Gênero', 'Região']])

# Concatenar features codificadas com as restantes
X_encoded = pd.DataFrame(X_encoded.toarray(), columns=encoder.get_feature_names_out(['Gênero', 'Região']), index=X.index)
X = pd.concat([X.drop(['Gênero', 'Região'], axis=1), X_encoded], axis=1)

# Codificar variável categórica 'IMC' com LabelEncoder
label_encoder = LabelEncoder()
X['Categoria IMC'] = label_encoder.fit_transform(X['Categoria IMC'])

# Substituir valores 'sim' e 'não' por 1 e 0
X['Fumante'] = X['Fumante'].map({'sim': 1, 'não': 0})

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

correlation_matrix = X_train.corr()
print('Visualizar a matriz de correlação:')
print(correlation_matrix)

Visualizar a matriz de correlação:
                     Idade       IMC    Filhos   Fumante  Categoria IMC  \
Idade             1.000000  0.031411  0.002567 -0.071434       0.044306   
IMC               0.031411  1.000000  0.131051 -0.025638      -0.301359   
Filhos            0.002567  0.131051  1.000000 -0.128791      -0.046793   
Fumante          -0.071434 -0.025638 -0.128791  1.000000      -0.096512   
Categoria IMC     0.044306 -0.301359 -0.046793 -0.096512       1.000000   
Região_numerica   0.030253  0.049917  0.010908  0.031080      -0.118367   
Gênero_masculino -0.145870  0.078598  0.008502 -0.135051      -0.084869   
Região_noroeste   0.040255  0.074665 -0.029174  0.040088      -0.067014   
Região_sudeste   -0.030567 -0.065390  0.001018  0.006848       0.141389   
Região_sudoeste  -0.003335  0.006233 -0.029174 -0.023530       0.047197   

                  Região_numerica  Gênero_masculino  Região_noroeste  \
Idade                    0.030253         -0.145870         0.04025

**4. Treinamento do Modelo**

In [8]:
# Adicionando mais modelos ao dicionário de modelos
models = {
    'Random Forest Regressor': RandomForestRegressor(),
    'Linear Regression': LinearRegression(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor()
}

# Treinamento e avaliação dos modelos
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    # Imprime uma linha separadora com base na largura do terminal
    print('-' * terminal_width)
    print(f'{name} MSE: {round(mse,2)}')
    # Calcular acurácia média usando validação cruzada
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f'Acurácia média com validação cruzada para {name}: {round(scores.mean(),2)}')
    
    # Calcula e exibe a importância das características para o modelo atual
    if hasattr(model, 'feature_importances_'):
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': model.feature_importances_})
        importance_df = importance_df.sort_values(by='Importância', ascending=False)
        print(f"Importância das características para {name}:")
        print(round(importance_df,2))
    elif isinstance(model, LinearRegression):
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Coeficiente': model.coef_})
        importance_df = importance_df.sort_values(by='Coeficiente', ascending=False)
        print(f"Coeficientes das características para {name}:")
        print(round(importance_df,2))
    elif isinstance(model, SVR):
        support_indices = model.support_
        support_features = X_train.iloc[support_indices]
        feature_importance_svr = support_features.mean(axis=0)
        importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': feature_importance_svr})
        importance_df = importance_df.sort_values(by='Importância', ascending=False)
        print(f"Importância das características para {name}:")
        print(round(importance_df,2))
    else:
        print(f"Não é possível calcular a importância das características para o modelo {name}")

--------------------------------------------------------------------------------------------------------------------------------------------------------------
Random Forest Regressor MSE: 163566233.27
Acurácia média com validação cruzada para Random Forest Regressor: 0.45
Importância das características para Random Forest Regressor:
            Colunas  Importância
0             Idade         0.57
1               IMC         0.27
2            Filhos         0.05
5   Região_numerica         0.03
3           Fumante         0.02
6  Gênero_masculino         0.02
9   Região_sudoeste         0.02
4     Categoria IMC         0.01
8    Região_sudeste         0.01
7   Região_noroeste         0.00
--------------------------------------------------------------------------------------------------------------------------------------------------------------
Linear Regression MSE: 178963033.65
Acurácia média com validação cruzada para Linear Regression: 0.52
Coeficientes das características para Lin

**5. Comparação de Modelos**

In [9]:
# Seleção de modelo com base na menor pontuação de validação cruzada
best_model_name = min(models, key=lambda model: cross_val_score(models[model], X_train, y_train, cv=5).mean())

**6. Seleção de Modelo**

In [10]:
best_model = models[best_model_name]
print(f'Best Model: {best_model_name}')

Best Model: Decision Tree Regressor


**7. Otimização de Modelo**

In [11]:
# Otimização de hiperparâmetros com Optuna (exemplo com RandomForestRegressor)
def objective(trial):
    if best_model_name == 'Random Forest Regressor':
        n_estimators = trial.suggest_int('n_estimators', 10, 100)
        max_depth = trial.suggest_int('max_depth', 2, 32)
        model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth)
    elif best_model_name == 'Decision Tree Regressor':
        max_depth = trial.suggest_int('max_depth', 2, 32)
        model = DecisionTreeRegressor(max_depth=max_depth)
    else:
        model = models[best_model_name]  # Use the default hyperparameters for other models
    score = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=3, scoring='neg_mean_squared_error').mean()
    return score

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-04-08 13:29:29,282] A new study created in memory with name: no-name-97940aca-590d-40e8-b92d-cba848862e3c
[I 2024-04-08 13:29:31,446] Trial 0 finished with value: -869878670.6998167 and parameters: {'max_depth': 4}. Best is trial 0 with value: -869878670.6998167.
[I 2024-04-08 13:29:32,983] Trial 1 finished with value: -975172690.6344585 and parameters: {'max_depth': 5}. Best is trial 1 with value: -975172690.6344585.
[I 2024-04-08 13:29:34,591] Trial 2 finished with value: -1038766779.0166645 and parameters: {'max_depth': 13}. Best is trial 2 with value: -1038766779.0166645.
[I 2024-04-08 13:29:34,620] Trial 3 finished with value: -1030056346.7813743 and parameters: {'max_depth': 16}. Best is trial 2 with value: -1038766779.0166645.
[I 2024-04-08 13:29:34,647] Trial 4 finished with value: -1154712960.7746308 and parameters: {'max_depth': 25}. Best is trial 4 with value: -1154712960.7746308.
[I 2024-04-08 13:29:34,675] Trial 5 finished with value: -1255516077.4135213 and parame

In [12]:
best_params = study.best_params
best_model.set_params(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Optimized {best_model_name} MSE: {round(mse,2)}')

Optimized Decision Tree Regressor MSE: 256625990.8


In [13]:
# Obter a importância das características apenas para modelos que suportam esse atributo
if hasattr(best_model, 'feature_importances_'):
    # Obter a importância das características do modelo
    feature_importances = best_model.feature_importances_
    
    # Criar um DataFrame para visualizar a importância das características
    importance_df = pd.DataFrame({'Colunas': X_train.columns, 'Importância': feature_importances})

    # Ordenar as características por importância
    importance_df = importance_df.sort_values(by='Importância', ascending=False)
    
    print("Quais características individuais têm maior impacto nos custos médicos cobrados pelo seguro de saúde?")
    print(round(importance_df,2))

    # Selecionar as características mais importantes (por exemplo, as 10 mais importantes)
    top_features = importance_df.head(10)['Colunas'].tolist()  # Ajuste o número conforme necessário
    
    # Calcular a matriz de correlação apenas para as características mais importantes
    correlation_matrix = X_train[top_features].corr()
    
    # Visualizar a matriz de correlação
    print("Existe alguma correlação entre certas características (por exemplo, idade, IMC) e os custos médicos?")
    print(correlation_matrix)
else:
    print("O modelo selecionado não suporta o cálculo de importância das características.")

Quais características individuais têm maior impacto nos custos médicos cobrados pelo seguro de saúde?
            Colunas  Importância
0             Idade         0.60
1               IMC         0.28
9   Região_sudoeste         0.04
8    Região_sudeste         0.03
2            Filhos         0.02
3           Fumante         0.02
5   Região_numerica         0.01
4     Categoria IMC         0.00
6  Gênero_masculino         0.00
7   Região_noroeste         0.00
Existe alguma correlação entre certas características (por exemplo, idade, IMC) e os custos médicos?
                     Idade       IMC  Região_sudoeste  Região_sudeste  \
Idade             1.000000  0.031411        -0.003335       -0.030567   
IMC               0.031411  1.000000         0.006233       -0.065390   
Região_sudoeste  -0.003335  0.006233         1.000000       -0.339340   
Região_sudeste   -0.030567 -0.065390        -0.339340        1.000000   
Filhos            0.002567  0.131051        -0.029174        0.001018

KeyError: 'Encargos'